In [ ]:
#import
import sys
from enum import Enum
import itertools
import pygame
import snakes.plugins
from learning.modules.process_mining.alpha_miner import AlphaMiner , Relations
from learning.modules.process_mining.alpha_plus import AlphaPlus
snakes.plugins.load('gv', 'snakes.nets', 'nets')
from nets import *
#just for the notebook
from IPython.display import Image
import pandas as pd


#extending ordering Relations

In [ ]:
class Relations_plus(Relations):
    SYMMETRIC       = '/\'
    XOR_JOIN 		=  '<|'
    XOR_SPLIT		= '|>'
    INDIRECTLY_FOLLOWED = '>>'

In [ ]:
# Defining the Alpha plus plus class
def __init__(self ,Traces):
        super().__init__(Traces)
        #traces within an event log
        self.traces = Traces
        # set of transitions that appear in loop of length one
        self.L1L = None
        # T' , traces minus L1L
        self.T_pr = None
        self.F_L1L = None
        self.Wm_L1L = SortedDict()
        self.alphaObject = None
    

    def extractRelations(self):
            #Extract non repetitive traces, alpha dont take care  about  frequencies !
        nnrep_traces = SortedSet()
        for trace in self.traces.values():
            nnrep_traces.add("".join(trace))
        print(nnrep_traces)
        #Extract relations between each transitions
        # generate Footprint
        for transition1 in self.transitions:
            self.relations[transition1] = SortedDict()
            for transition2 in self.transitions:
                concat = transition1+transition2
                concat_symetric_1 = transition1+transition2+transition1
                concat_symetric_2 = transition2+transition1+transition2

                print(concat)
                print(concat_symetric_1)
                print(concat_symetric_2)

                relation = None
                for trace in nnrep_traces:
                    
                    if relation == None :
                        if trace.find(concat) >= 0:
                            relation = Relations.RIGHT_CAUSALITY

                        elif trace.find(concat[::-1]) >= 0:
                            relation = Relations.LEFT_CAUSALITY
                        elif trace.find(concat_symetric_1):
                            relation = Relations_plus.SYMMETRIC
                    else:
                        if trace.find(concat) >= 0:
                            if relation == Relations.LEFT_CAUSALITY:
                                if trace.find(concat_symetric_1) <= 0 and trace.find(concat_symetric_2) <= 0:
                                    relation = Relations.PARALLEL
                        elif trace.find(concat[::-1]) >= 0:
                            if relation == Relations.RIGHT_CAUSALITY:
                                if trace.find(concat_symetric_1) <= 0 and trace.find(concat_symetric_2) <= 0:
                                    relation = Relations.PARALLEL  
                                           

                if relation == None:
                    relation = Relations.CHOICES
                self.relations[transition1][transition2] = relation


        return self.relations

    def extract_L1L(self):
        #extract length one loop 
        self.L1L = SortedSet()
        super().getTransitions()
        #compute footprint and extract all transitions that have a causality relations with himself
        print(self.transitions)
        self.extractRelations()
        print(self.relations)
        for transition in self.transitions:
            if self.relations[transition][transition] == Relations.PARALLEL:
                self.L1L.add(transition)
        return self.L1L
    def extract_Tprime(self):
        # T' := T \ L1L
        self.T_pr = self.transitions.difference(self.L1L)
        return self.T_pr